# RFM Analysis - Customer Segmentation

This notebook performs an RFM (Recency, Frequency, Monetary) analysis on sales data to segment customers based on their purchasing behavior:
- **Recency**: How recently did the customer purchase?
- **Frequency**: How often do they purchase?
- **Monetary**: How much do they spend?

These three metrics help identify high-value customers, customers at risk of churn, and other important segments.

In [ ]:
# Install required libraries
!pip install pandas numpy matplotlib seaborn plotly

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings('ignore')
plt.style.use('ggplot')
%matplotlib inline

## 1. Data Loading and Preparation

In [ ]:
# Load the sales data by first downloading the raw bytes and then decoding properly
import requests
import io

url = "https://pocs.nyc3.cdn.digitaloceanspaces.com/sales_data_sample.csv"

# Download the raw bytes
response = requests.get(url)
raw_data = response.content

# First, try to detect encoding - most likely latin1 or cp1252 based on the error
try:
    decoded_content = raw_data.decode('latin1')
    encoding_used = 'latin1'
except UnicodeDecodeError:
    try:
        decoded_content = raw_data.decode('cp1252')
        encoding_used = 'cp1252'
    except UnicodeDecodeError:
        # Fallback to a very permissive encoding that rarely fails
        decoded_content = raw_data.decode('latin1', errors='replace')
        encoding_used = 'latin1 with replacement'

print(f"Successfully decoded file using {encoding_used} encoding")

# Load the CSV from the properly decoded string
sales_data = pd.read_csv(io.StringIO(decoded_content))

# Display the first few rows
sales_data.head()

In [ ]:
# Check the data shape and information
print(f"Dataset Shape: {sales_data.shape}")
print("\nData Types:")
sales_data.info()

In [ ]:
# Check for missing values
missing_values = sales_data.isnull().sum()
print("Missing Values:")
print(missing_values[missing_values > 0] if len(missing_values[missing_values > 0]) > 0 else "No missing values")

In [ ]:
# Convert ORDERDATE to datetime format
sales_data['ORDERDATE'] = pd.to_datetime(sales_data['ORDERDATE'])

# Filter out canceled orders if present
sales_data = sales_data[sales_data['STATUS'] != 'Cancelled']

## 2. RFM Metrics Calculation

In [ ]:
# Get the most recent date in the dataset
max_date = sales_data['ORDERDATE'].max()
print(f"Most recent order date: {max_date}")

# Create analysis date (1 day after the most recent order)
analysis_date = max_date + timedelta(days=1)
print(f"Analysis date: {analysis_date}")

In [ ]:
# Group by customer and calculate RFM metrics
rfm = sales_data.groupby('CUSTOMERNAME').agg({
    'ORDERDATE': lambda x: (analysis_date - x.max()).days,  # Recency
    'ORDERNUMBER': 'nunique',  # Frequency
    'SALES': 'sum'  # Monetary
}).reset_index()

# Rename columns
rfm.rename(columns={
    'ORDERDATE': 'Recency',
    'ORDERNUMBER': 'Frequency', 
    'SALES': 'Monetary'
}, inplace=True)

# Display the first few rows
rfm.head()

In [ ]:
# Summary statistics
rfm.describe()

## 3. RFM Scoring

In [ ]:
# Create RFM scores with robust error handling

# For Recency - lower is better (more recent purchases)
try:
    rfm['R_Score'] = pd.qcut(rfm['Recency'], q=5, labels=[5, 4, 3, 2, 1])
except ValueError as e:
    print(f"Warning for Recency scoring: {e}")
    # Alternative approach - use rank method
    rfm['R_Rank'] = rfm['Recency'].rank(ascending=True)
    rfm['R_Score'] = pd.cut(
        rfm['R_Rank'], 
        bins=[0, rfm['R_Rank'].max()*0.2, rfm['R_Rank'].max()*0.4, 
              rfm['R_Rank'].max()*0.6, rfm['R_Rank'].max()*0.8, rfm['R_Rank'].max()], 
        labels=[5, 4, 3, 2, 1],
        include_lowest=True
    )
    
# For Frequency - higher is better
try:
    # First, check how many unique values we have
    unique_freq = rfm['Frequency'].nunique()
    print(f"Number of unique frequency values: {unique_freq}")
    
    # If we have limited unique values, adjust the number of quantiles
    if unique_freq < 5:
        print(f"Warning: Only {unique_freq} unique frequency values, using {unique_freq} quantiles instead of 5")
        # Use regular cut with manually determined bins
        freq_values = sorted(rfm['Frequency'].unique())
        # Create labels based on number of unique values
        freq_labels = list(range(1, unique_freq + 1))
        rfm['F_Score'] = pd.cut(rfm['Frequency'], bins=[-1] + freq_values, labels=freq_labels)
    else:
        rfm['F_Score'] = pd.qcut(rfm['Frequency'], q=5, labels=[1, 2, 3, 4, 5], duplicates='drop')
except ValueError as e:
    print(f"Warning for Frequency scoring: {e}")
    # Alternative approach - use rank method
    rfm['F_Rank'] = rfm['Frequency'].rank(ascending=False)
    rfm['F_Score'] = pd.cut(
        rfm['F_Rank'], 
        bins=[0, rfm['F_Rank'].max()*0.2, rfm['F_Rank'].max()*0.4, 
              rfm['F_Rank'].max()*0.6, rfm['F_Rank'].max()*0.8, rfm['F_Rank'].max()], 
        labels=[1, 2, 3, 4, 5],
        include_lowest=True
    )

# For Monetary - higher is better
try:
    rfm['M_Score'] = pd.qcut(rfm['Monetary'], q=5, labels=[1, 2, 3, 4, 5])
except ValueError as e:
    print(f"Warning for Monetary scoring: {e}")
    # Alternative approach - use rank method
    rfm['M_Rank'] = rfm['Monetary'].rank(ascending=False)
    rfm['M_Score'] = pd.cut(
        rfm['M_Rank'], 
        bins=[0, rfm['M_Rank'].max()*0.2, rfm['M_Rank'].max()*0.4, 
              rfm['M_Rank'].max()*0.6, rfm['M_Rank'].max()*0.8, rfm['M_Rank'].max()], 
        labels=[1, 2, 3, 4, 5],
        include_lowest=True
    )

# Calculate overall RFM Score
rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

# Convert to numeric for segmentation
rfm['RFM_Score_Numeric'] = rfm['R_Score'].astype(int) + rfm['F_Score'].astype(int) + rfm['M_Score'].astype(int)

# Display the first few rows with scores
rfm.head()

## 4. Customer Segmentation

In [ ]:
# Define RFM segments
def rfm_segment(r, f, m):
    if r >= 4 and f >= 4 and m >= 4:
        return 'Champions'
    elif (r >= 2 and r <= 4) and (f >= 3 and f <= 4) and (m >= 3):
        return 'Loyal Customers'
    elif (r >= 3 and r <= 5) and (f >= 1 and f <= 3) and (m >= 1 and m <= 3):
        return 'Potential Loyalists'
    elif r >= 4 and (f >= 0 and f <= 1) and (m >= 0 and m <= 1):
        return 'New Customers'
    elif (r >= 3 and r <= 4) and (f >= 0 and f <= 1) and (m >= 0 and m <= 1):
        return 'Promising'
    elif (r >= 2 and r <= 3) and (f >= 2 and f <= 3) and (m >= 2 and m <= 3):
        return 'Customers Needing Attention'
    elif (r >= 2 and r <= 3) and (f >= 0 and f <= 2) and (m >= 0 and m <= 2):
        return 'About To Sleep'
    elif r <= 2 and f >= 2 and m >= 2:
        return 'At Risk'
    elif r <= 1 and (f >= 4 and f <= 5) and (m >= 4 and m <= 5):
        return 'Cant Lose Them'
    elif (r >= 1 and r <= 2) and (f >= 1 and f <= 2) and m >= 2:
        return 'Hibernating'
    else:
        return 'Lost'
    
# Apply the segmentation function
rfm['Segment'] = rfm.apply(lambda x: rfm_segment(x['R_Score'], x['F_Score'], x['M_Score']), axis=1)

# Display segment counts
segment_counts = rfm['Segment'].value_counts().reset_index()
segment_counts.columns = ['Segment', 'Count']
segment_counts

## 5. RFM Analysis Visualizations

In [ ]:
# Plot the distribution of customer segments
plt.figure(figsize=(12, 6))
sns.barplot(x='Segment', y='Count', data=segment_counts.sort_values('Count', ascending=False))
plt.title('Customer Segments Distribution', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Calculate average metrics by segment
segment_avg = rfm.groupby('Segment').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean',
    'CUSTOMERNAME': 'count'
}).reset_index()

segment_avg = segment_avg.rename(columns={'CUSTOMERNAME': 'Count'})
segment_avg = segment_avg.sort_values('Count', ascending=False)

# Round the values for better readability
segment_avg['Recency'] = segment_avg['Recency'].round(1)
segment_avg['Frequency'] = segment_avg['Frequency'].round(1)
segment_avg['Monetary'] = segment_avg['Monetary'].round(2)

segment_avg

In [ ]:
# Create a heatmap of segment performance
try:
    plt.figure(figsize=(12, 8))
    
    # Print the dataframe structure to help debug
    print("Segment average dataframe structure:")
    print(segment_avg.columns)
    print(segment_avg.head(2))
    
    # Method 1: Try using pivot
    try:
        # Create a proper pivot table without the None column parameter
        segment_pivot = segment_avg.pivot(index='Segment', values=['Recency', 'Frequency', 'Monetary'])
        print("\nPivot successful")
    except Exception as e1:
        print(f"\nPivot failed: {e1}")
        # Method 2: Alternative approach - just use the dataframe directly
        try:
            segment_pivot = segment_avg.set_index('Segment')[['Recency', 'Frequency', 'Monetary']]
            print("\nAlternative method successful")
        except Exception as e2:
            print(f"\nAlternative method failed: {e2}")
            # Method 3: Last resort - create a new dataframe manually
            segment_data = []
            for segment in segment_avg['Segment'].unique():
                row = segment_avg[segment_avg['Segment'] == segment].iloc[0]
                segment_data.append({
                    'Segment': segment,
                    'Recency': row['Recency'] if 'Recency' in row else 0,
                    'Frequency': row['Frequency'] if 'Frequency' in row else 0,
                    'Monetary': row['Monetary'] if 'Monetary' in row else 0
                })
            segment_pivot = pd.DataFrame(segment_data).set_index('Segment')
            print("\nManual creation method successful")
    
    # Check if we have a valid dataframe to plot
    print("\nFinal dataframe for visualization:")
    print(segment_pivot.head(2))
    
    # Plot the heatmap
    ax = sns.heatmap(segment_pivot, annot=True, cmap='YlGnBu', fmt='.1f')
    plt.title('Segment Performance Heatmap', fontsize=16)
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Error creating heatmap: {e}")
    print("\nFalling back to simple tabular display:")
    display(segment_avg[['Segment', 'Recency', 'Frequency', 'Monetary']].sort_values('Segment'))

In [ ]:
# Visualize the 3D RFM space
fig = px.scatter_3d(rfm, x='Recency', y='Frequency', z='Monetary',
                   color='Segment', hover_name='CUSTOMERNAME',
                   opacity=0.7, width=900, height=700)
fig.update_layout(title='3D RFM Segmentation')
fig.show()

## 6. Deeper Analysis by Segment

In [ ]:
# Merge the original sales data with the RFM segments
sales_with_segment = pd.merge(sales_data, rfm[['CUSTOMERNAME', 'Segment']], on='CUSTOMERNAME', how='left')

# Group by segment and calculate metrics
segment_sales = sales_with_segment.groupby('Segment').agg({
    'SALES': 'sum',
    'ORDERNUMBER': 'nunique',
    'CUSTOMERNAME': 'nunique'
}).reset_index()

segment_sales.columns = ['Segment', 'Total Sales', 'Total Orders', 'Customer Count']
segment_sales['Avg Order Value'] = segment_sales['Total Sales'] / segment_sales['Total Orders']
segment_sales['Avg Customer Value'] = segment_sales['Total Sales'] / segment_sales['Customer Count']

# Round the values
segment_sales['Avg Order Value'] = segment_sales['Avg Order Value'].round(2)
segment_sales['Avg Customer Value'] = segment_sales['Avg Customer Value'].round(2)
segment_sales['Total Sales'] = segment_sales['Total Sales'].round(2)

# Sort by total sales
segment_sales = segment_sales.sort_values('Total Sales', ascending=False)
segment_sales

In [ ]:
# Visualize the average customer value by segment
plt.figure(figsize=(12, 6))
sns.barplot(x='Segment', y='Avg Customer Value', data=segment_sales.sort_values('Avg Customer Value', ascending=False))
plt.title('Average Customer Value by Segment', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Visualize the percentage of total sales by segment
plt.figure(figsize=(10, 10))
segment_sales['Sales Percentage'] = segment_sales['Total Sales'] / segment_sales['Total Sales'].sum() * 100
plt.pie(segment_sales['Sales Percentage'], labels=segment_sales['Segment'], autopct='%1.1f%%', 
        startangle=90, shadow=True, explode=[0.05]*len(segment_sales))
plt.title('Percentage of Total Sales by Customer Segment', fontsize=16)
plt.axis('equal')
plt.show()

## 7. Product Line Analysis by Segment

In [ ]:
# Analyze product preferences by segment
product_segment = sales_with_segment.groupby(['Segment', 'PRODUCTLINE']).agg({
    'SALES': 'sum',
    'ORDERNUMBER': 'nunique'
}).reset_index()

# Sort by segment and sales
product_segment = product_segment.sort_values(['Segment', 'SALES'], ascending=[True, False])
product_segment

In [ ]:
# Create a pivot table for better visualization
product_pivot = sales_with_segment.pivot_table(index='Segment', 
                                              columns='PRODUCTLINE', 
                                              values='SALES', 
                                              aggfunc='sum')

# Visualize the product preferences
plt.figure(figsize=(14, 10))
sns.heatmap(product_pivot, annot=True, fmt='.0f', cmap='viridis')
plt.title('Product Line Preferences by Customer Segment', fontsize=16)
plt.tight_layout()
plt.show()

## 8. Regional Analysis by Segment

In [ ]:
# Analyze regional distribution by segment
region_segment = sales_with_segment.groupby(['Segment', 'COUNTRY']).agg({
    'CUSTOMERNAME': 'nunique',
    'SALES': 'sum'
}).reset_index()

# Sort by segment and number of customers
region_segment = region_segment.sort_values(['Segment', 'CUSTOMERNAME'], ascending=[True, False])
region_segment.columns = ['Segment', 'Country', 'Customer Count', 'Total Sales']

# Get top 5 countries by segment
def get_top_n(group, n=3):
    return group.nlargest(n, 'Customer Count')

top_countries = region_segment.groupby('Segment').apply(get_top_n).reset_index(drop=True)
top_countries

In [ ]:
# Plot top countries for selected segments
important_segments = ['Champions', 'Loyal Customers', 'At Risk', 'Cant Lose Them']
filtered_data = top_countries[top_countries['Segment'].isin(important_segments)]

plt.figure(figsize=(14, 8))
chart = sns.barplot(x='Country', y='Customer Count', hue='Segment', data=filtered_data)
plt.title('Top Countries by Important Customer Segments', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.legend(title='Segment')
plt.tight_layout()
plt.show()

## 9. RFM Segment Recommendations

In [ ]:
# Create a dataframe with segment descriptions and recommended actions
segment_descriptions = pd.DataFrame({
    'Segment': [
        'Champions', 'Loyal Customers', 'Potential Loyalists', 'New Customers',
        'Promising', 'Customers Needing Attention', 'About To Sleep',
        'At Risk', 'Cant Lose Them', 'Hibernating', 'Lost'
    ],
    'Description': [
        'Best customers who buy often and spend the most',
        'Regular shoppers with above-average monetary value',
        'Recent customers with moderate frequency, could become loyal',
        'Customers who recently made their first purchases',
        'Recent shoppers who haven\'t spent much',
        'Average recency, frequency, and monetary values',
        'Below average recency and frequency, at risk of losing',
        'Once valuable customers who haven\'t purchased recently',
        'Made big purchases a long time ago, but haven\'t returned recently',
        'Low recency and frequency, but above average monetary value',
        'Lowest recency, frequency, and monetary values'
    ],
    'Recommended Actions': [
        'Reward, upsell premium products, seek reviews, create brand ambassadors',
        'Engage with loyalty programs, personalized communication, exclusive offers',
        'Targeted offers, early product access, encourage more frequent purchases',
        'Welcome series, education on value proposition, first-time buyer promotions',
        'Provide first-purchase incentives, build relationship, educational content',
        'Reactivation campaigns, satisfaction surveys, personalized recommendations',
        'Reactivation with higher incentives, remind of value proposition',
        'Winback campaigns, deep discounts, re-engagement content',
        'Reactivation outreach, special renewal offers, VIP treatment',
        'Reactivation campaigns, remind of past purchases, new product alerts',
        'Very deep discounts or remove from marketing list to save costs'
    ]
})

segment_descriptions

## 10. Export Top Customers by Segment

In [ ]:
# Get top 5 customers by monetary value in each segment
top_customers = rfm.sort_values(['Segment', 'Monetary'], ascending=[True, False])
top_segment_customers = top_customers.groupby('Segment').head(5)

# Select relevant columns for the final report
top_segment_customers = top_segment_customers[['CUSTOMERNAME', 'Segment', 'Recency', 'Frequency', 'Monetary']]
top_segment_customers.sort_values(['Segment', 'Monetary'], ascending=[True, False])

## 11. RFM Executive Summary

Based on our RFM analysis, we've segmented the customer base into different groups based on their purchasing behavior. Here are the key findings:

1. **Champions** - These are our best customers, who buy often and spend the most. They should be prioritized for rewards, special offers, and referral programs.

2. **At Risk Customers** - We've identified a segment of once valuable customers who haven't purchased recently. These need immediate attention with winback campaigns.

3. **Potential Loyalists** - Recent customers with moderate frequency who could be converted to loyal customers with the right approach.

4. **Product Preferences** - Different segments show distinct product preferences that can be leveraged for targeted marketing campaigns.

5. **Regional Distribution** - Customer segments are not evenly distributed across regions, suggesting opportunity for region-specific strategies.

**Recommendations:**
- Implement segment-specific marketing strategies based on the recommendations provided
- Create automated workflows to maintain customer segment information up-to-date
- Develop dashboards to track segment migration over time
- Design experiments to test effectiveness of segment-specific marketing actions